In [12]:
# ------- SERVER EXTENSIONS ---------
lib =  r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities/droplet_dataset'
lib2 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/utilities'
lib3 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/data_analysis'
lib4 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy'
lib5 = r'/srv01/technion/shitay/Code/classifying_response_to_immunotherapy/scripts'
import sys
sys.path.append(lib)
sys.path.append(lib2)
sys.path.append(lib3)
sys.path.append(lib4)
sys.path.append(lib5)
# ------- SERVER EXTENSIONS ---------
import numpy as np
import pandas as pd
import scipy
import sklearn
from sklearn.manifold import TSNE
import pickle
# from Bio.Cluster import kcluster
import os
import numpy as np
import yaml
import os
import math
import pandas
from collections import Counter
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
# from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import sys
import seaborn as sns
# import pyclustering
import matplotlib.pylab as plt
import seaborn as sb
from utilities.general_helpers import *
from os.path import join
from utilities.droplet_dataset import get_requested_subset
from shutil import copyfile
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
from tqdm import tqdm

Loads data:

In [2]:
FILTERED_CELLS_PATH = r'/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cohort_normalized_26.6.21_protein_coding_genes.pkl'
print(f'Loading cohort file:\n{FILTERED_CELLS_PATH}')
cohort = pickle.load(open(FILTERED_CELLS_PATH, 'rb'))
filtered_cells = cohort.filter_cells_by_property('is_immune', True)

Loading cohort file:
/storage/md_keren/shitay/Data/droplet_seq/cohort/normalized/6.21/cohort_normalized_26.6.21_protein_coding_genes.pkl


Classifier func:

In [3]:
class Classifier(nn.Module):

    def __init__(self, output_dim=None, input_size=None):
        """

        The Classifer class: We are developing a model similar to ACTINN for good accuracy

        """
        if output_dim == None or input_size == None:
            raise ValueError('Must explicitly declare input dim (num features) and output dim (number of classes)')

        super(Classifier, self).__init__();
        self.inp_dim = input_size;
        self.out_dim = output_dim;

        # feed forward layers
        self.classifier_sequential = nn.Sequential(
                                        nn.Linear(self.inp_dim, 100),
                                        nn.ReLU(),

                                        nn.Linear(100, 50),
                                        nn.ReLU(),

                                        nn.Linear(50, 25),
                                        nn.ReLU(),

                                        nn.Linear(25, output_dim)
                                        # SoftMax not needed for CrossEntropyLoss in PyTorch
                                        ## i.e. no need for nn.Softmax(dim=1)
                                        )

    def forward(self, x):
        """

        Forward pass of the classifier

        """

        out = self.classifier_sequential(x);

        return out

Dataset object:

In [4]:
def get_response_labels():
    # Loads xlsx files
        CLINICAL_LABELS_PATH = r'/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx'
        MELANOMA_CLINICAL_DATA_PATH = r'/storage/md_keren/shitay/Data/tables/edited_unprotected_Melanoma_clinical_data.xlsx'
        melanoma_clinical_data = pd.read_excel(MELANOMA_CLINICAL_DATA_PATH)
        clinical_labels = pd.read_excel(CLINICAL_LABELS_PATH)

        # takes nly first 46 samples, fill Nan and creat dictionary mapping
        melanoma_clinical_data = melanoma_clinical_data.iloc[:46,:][['Patient id', 'Clinical response']]
        melanoma_clinical_data['Clinical response'] = melanoma_clinical_data['Clinical response'].fillna('??')
        labels_mapping = {v[0]: v[1] for v in clinical_labels[['Clinical response', 'binary label']].values}

        # adds CRITERIA response into clinical df
        melanoma_clinical_data['response'] = melanoma_clinical_data['Clinical response'].apply(lambda x: labels_mapping[x])
        res_melanoma_clinical_data = melanoma_clinical_data.set_index('Patient id')
        res_melanoma_clinical_data = [res_melanoma_clinical_data.loc[sample]['response'] for sample in cohort.samples]
        return [0 if xx=='NR' else 1 for xx in res_melanoma_clinical_data]
r_labels =  get_response_labels()  

In [5]:
GENES = 20000
CELLS = 79000

In [6]:

class SCDataset(Dataset):
    def __init__(self, cohort, transform=None, target_transform=None):       
        self.X = torch.Tensor(cohort.counts[:CELLS, :GENES])
        self.Y = r_labels[:CELLS]
        self.transform = transform
        self.target_transform = target_transform

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        X = self.X[idx]
        label = self.Y[idx]
        if self.transform:
            X = self.transform(X)
        if self.target_transform:
            label = self.target_transform(label)
        return X, label

In [7]:
dataset = SCDataset(filtered_cells)

train_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

Builds a classifier, Training optimization and loss function:

In [8]:
classifier = Classifier(output_dim=2, input_size=GENES)

cf_criterion = torch.nn.CrossEntropyLoss()

#     cf_criterion = torch.nn.BCEWithLogitsLoss()
cf_optimizer = torch.optim.Adam(params=classifier.parameters(), 
                                lr=0.0001, 
                                betas=(0.9, 0.999), 
                                eps=1e-08, 
                                weight_decay=0.005, 
                                amsgrad=False)
cf_decayRate = 0.95
cf_lr_scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer=cf_optimizer, gamma=cf_decayRate)
print("\n Classifier Model \n")
print(classifier)



 Classifier Model 

Classifier(
  (classifier_sequential): Sequential(
    (0): Linear(in_features=20000, out_features=100, bias=True)
    (1): ReLU()
    (2): Linear(in_features=100, out_features=50, bias=True)
    (3): ReLU()
    (4): Linear(in_features=50, out_features=25, bias=True)
    (5): ReLU()
    (6): Linear(in_features=25, out_features=2, bias=True)
  )
)


Trains:

In [9]:
def accuracy():
    all_data_x = dataset[:][0]
    all_data_y = dataset[:][1]
    # all_data_y
    res = classifier(all_data_x).detach().numpy().argmax(1) == np.array(all_data_y)
    return sum(res)/len(res)

print(f'Accuracy before train: {accuracy()}')


Accuracy before train: 0.7481139240506329


In [10]:
ClassifierEpochs = 10


def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
#         print(X.shape)
#         print(X)
#         print(y)
        
        pred = model(X)
        loss = loss_fn(pred, y.type(torch.long))

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


for epoch in tqdm(range(0, ClassifierEpochs + 1), desc="Classifier Training"): 

    classifier.train()
    train_loop(train_dataloader, classifier, cf_criterion, cf_optimizer) 
    
print(f'Accuracy after train: {accuracy()}')


Classifier Training:   0%|          | 0/11 [00:00<?, ?it/s]/Local/md_keren/anaconda3/lib/python3.8/site-packages/torch/autograd/__init__.py:130: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  Variable._execution_engine.run_backward(
Classifier Training: 100%|██████████| 11/11 [10:05<00:00, 55.00s/it]


Accuracy after train: 0.993126582278481


In [14]:
# all_data_x = dataset[:][0]
# all_data_y = dataset[:][1]
# # all_data_y
# res = classifier(all_data_x).detach().numpy().argmax(1) == np.array(all_data_y)
# dataset[:]

# Debug

In [502]:
cf_criterion(torch.Tensor([[.1,10.12,.7]]), 
             torch.Tensor([1]).type(torch.long))
# cf_criterion
# torch.Tensor([[1]]).type(torch.int)

tensor(0.0001)

In [319]:
loss = nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
loss(input, target)

tensor(2.3610, grad_fn=<NllLossBackward>)

In [497]:
input.dtype
# target.dtype
classifier(torch.randn(3, 20000, requires_grad=True))

tensor([[ 7.9772, -7.9510],
        [ 4.8489, -4.9263],
        [ 9.2736, -9.2079]], grad_fn=<AddmmBackward>)

In [ ]:
# # # Display image and label.
# train_features, train_labels = next(iter(train_dataloader))
# print(f"Feature batch shape: {train_features}")
# print(f"Labels batch shape: {train_labels}")
# # img = train_features[0]
# # label = train_labels[0]
# # plt.imshow(img, cmap="gray")
# # plt.show()
# # print(f"Label: {label}")

In [507]:

# Loads xlsx files
CLINICAL_LABELS_PATH = r'/storage/md_keren/shitay/Data/tables/clinical_labels.xlsx'
MELANOMA_CLINICAL_DATA_PATH = r'/storage/md_keren/shitay/Data/tables/edited_unprotected_Melanoma_clinical_data.xlsx'
melanoma_clinical_data = pd.read_excel(MELANOMA_CLINICAL_DATA_PATH)
clinical_labels = pd.read_excel(CLINICAL_LABELS_PATH)

# takes nly first 46 samples, fill Nan and creat dictionary mapping
melanoma_clinical_data = melanoma_clinical_data.iloc[:46,:][['Patient id', 'Clinical response']]
melanoma_clinical_data['Clinical response'] = melanoma_clinical_data['Clinical response'].fillna('??')
labels_mapping = {v[0]: v[1] for v in clinical_labels[['Clinical response', 'binary label']].values}

# adds CRITERIA response into clinical df
melanoma_clinical_data['response'] = melanoma_clinical_data['Clinical response'].apply(lambda x: labels_mapping[x])
res_melanoma_clinical_data = melanoma_clinical_data.set_index('Patient id')
# res_melanoma_clinical_data = [res_melanoma_clinical_data.loc[sample]['response'] for sample in cohort.samples]



In [511]:
pd.read_excel(MELANOMA_CLINICAL_DATA_PATH)

,#,Patient id,MGH id,Date collected,Sex (F/M),Age (relevant to time of sample),Melanoma type,Primary=1; Metastasis=0,Genotype,Therapy(ies) prior to biopsy,Therapy after biopsy,Clinical response,Status (Alive=1/Dead=0),Biopsy site,"Biopsy type (pre, post)",Overall survival (days),Comments,Cell line (Y; N),TILs (Y; N)
0,1,M97,1256,2019-10-11,M,79.0,Cutaneous,0,TP63; DDR2; ATM; MAP2K1; HNF1A; TERT,NaN,PD1 (started on 3/31/20),R,1.0,Left Axillary LN,Pre,NaN,NaN,NaN,NaN
1,2,M98,509,2019-10-11,M,50.0,Cutaneous,0,NaN,"aPD1 (progressed on 7/29/19)- 18 cycles, start...",NA (surgical management),R (had a response in 2016-2019),1.0,Right inguinal LN- progressing lesion,Post,NaN,1126 days- PFS,NaN,NaN
2,3,M99,1063,2019-10-11,F,69.0,Mucosal,0,"NF2, NRAS, CTNNB1 variants",5/11/17- ipi+nivo; 7/18/17- nivo; 1/8/18- pemb...,TVEC,NR (PD),1.0,Left distal femur curettage,Post,NaN,NaN,NaN,NaN
3,4,M100,592,2019-10-21,M,88.0,Cutaneous,0,"NRAS, TERT, RET, NF1, SMARCA4 variants",1/3/19- pembro (one dose-toxicity),NaN,NR (SD),1.0,Endoscopy,Post,NaN,NaN,NaN,NaN
4,5,M101,1259,2019-10-21,M,57.0,Unknown,0,"TRET, TP53, NRAS variants",NaN,Pembro (adj)- 1/14/20,NED- 10/5/20,1.0,Left inguinal LN (first met diagnosis),Pre adj,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,87,M190,1608,2021-03-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87,88,M191,1571,2021-03-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,89,M192,1487,2021-03-25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
89,90,M193,1622,2021-03-26,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
